<a href="https://colab.research.google.com/github/memo-math/Proyecto-Final-CienciadeDatos-CF/blob/main/XG_carga_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creación de un modelo de goles esperados 🚀 ⚽ 🧙

Desde hace tiempo la ciencia de datos se aplica al mundo del deporte, quizá el caso más famoso de su uso es el mostrado por la película [Moneyball](https://www.imdb.com/title/tt1210166/?ref_=nv_sr_srsg_0) donde los atleticos de Oakland, a pesar de su bajo presupuesto, logran hacer una gran temporada. 

Para este proyecto final del Bootcamp de Ciencia de Datos de Código Facilito aplicaremos lo aprendido al fútbol, donde se pueden hacer cosas como analizar los pases y su efectividad, ver la geometría del control del campo de distintos equipos, analizar el recorrido por el campo de juego de distintos jugadores, etc.

Para este trabajo nos enfocaremos en la creación de un modelo de goles esperados.

## ¿Qué son los goles esperados?

Sin duda uno de lo eventos más importantes de un partido de fútbol son los goles, ya que estos determinan al equipo ganador. Todo gol viene precedido de un tiro, los goles esperados(xG) determinan la calidad de los tiros, es decir, expresan la probabilidad de que un tiro se convierta en gol. Para poder deducir los xG se pueden utilizar distintos factores como el tipo de tiro, la parte del cuerpo, la posición desde la que se realizo el tiro dentro del campo, etc. 

Para nuestro modelo los factores que tendremos en cuenta están limitados por los datos abiertos que usaremos. Sin embargo, de acuerdo a diversas fuentes, estos datasets contienen los features más relevantes

## Los datos

Los datasets que usaremos se han obtenido de  los datos abiertos de [statsbomb](https://github.com/statsbomb/open-data). En particular, tomaremos la información del fútbol femenil. Se encuentran disponibles tres temporadas de la liga inglesa (FA Women´s Super League) y una temporada de la liga de estados unidos (NWSL).

De acuerdo con la documentación los datos se proveen en archivos JSON, y para poder acceder a los que nos interesan primero debemos tener los id's de las competiciones y temporadas. En este caso, se obtuvieron manualmente dichos id's del archivo [competitions.json](https://github.com/statsbomb/open-data/blob/master/data/competitions.json).

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
competition_id = [37, 49]  #id: 37 -> FA Women´s Super League, id: 49 -> NWSL
seasons_fa_id = [90, 42, 4] #id's de las temporadas disponibles de la FA Women´s Super League 
seasons_nwsl_id = [3] #id de la temporada disponible de la NWSL

Con los anteriores id's se pueden obtener las urls que tienen los id's de los partidos que vamos a analizar

In [ ]:
matches_urls = []
url_matches_fa = 'https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches'  
for season in seasons_fa_id:
  complete_url = url_matches_fa + '/' + str(competition_id[0]) + '/' + str(season) + '.json'
  matches_urls.append(complete_url)
url_matches_usa = 'https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches' + '/' + str(competition_id[1]) + '/' + str(seasons_nwsl_id[0]) + '.json'
matches_urls.append(url_matches_usa)

Para cada una de las urls leeremos el archivo json y de él extraeremos los id's de los partidos:

In [ ]:
matches_ids = []
for url in matches_urls:
  df = pd.read_json(url)
  matches_ids.append(df['match_id'].values)

Hasta el momento matches_ids es una lista de listas que tiene los id's de los partidos. Procedamos a convertirla a solo una lista.

In [ ]:
flat_match_id_list = [item for sublist in matches_ids for item in sublist]

Ahora ya podemos descargar los datos de cada uno de los partidos.

In [ ]:
#Primero obtenemos las urls de los eventos de cada partido
events_url_list = []
url_events = 'https://raw.githubusercontent.com/statsbomb/open-data/master/data/events'
for matchid in flat_match_id_list:
  url_events_match = url_events + '/' + str(matchid) + '.json'
  events_url_list.append(url_events_match)

In [ ]:
#leemos los archivos json de las urls y guardamos cada uno de los archivos en una lista
events_list=[]
for url in events_url_list:
  events_list.append(pd.read_json(url))

Como cada elemento de la lista es ahora un dataframe debemos juntarlos en un solo dataframe

In [ ]:
events_df = pd.concat(events_list, ignore_index=True)

Este nuevo dataframe contiene la información de todos los partidos seleccionados. Cada fila del dataframe es un evento que ocurrió durante el partido. Un evento del partido puede ser la alineación, el inicio del partido, un pase, un tiro, etc.

In [ ]:
events_df.head(3)

,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,team,duration,tactics,related_events,player,position,location,pass,carry,under_pressure,ball_receipt,counterpress,duel,dribble,goalkeeper,clearance,shot,foul_committed,out,interception,off_camera,ball_recovery,foul_won,block,substitution,50_50,injury_stoppage,miscontrol,bad_behaviour,half_start,half_end,player_off
0,3ae5d153-60b6-4fb0-a4c3-7406eb19134b,1,1,2022-03-03,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 2647, 'name': 'Aston Villa'}","{'id': 1, 'name': 'Regular Play'}","{'id': 2647, 'name': 'Aston Villa'}",0.0,"{'formation': 352, 'lineup': [{'player': {'id'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dd387270-fb96-4574-aba5-4817cd6de379,2,1,2022-03-03,0,0,"{'id': 35, 'name': 'Starting XI'}",1,"{'id': 2647, 'name': 'Aston Villa'}","{'id': 1, 'name': 'Regular Play'}","{'id': 968, 'name': 'Arsenal WFC'}",0.0,"{'formation': 433, 'lineup': [{'player': {'id'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1cae007e-b10b-4959-be1f-5b595313b72c,3,1,2022-03-03,0,0,"{'id': 18, 'name': 'Half Start'}",1,"{'id': 2647, 'name': 'Aston Villa'}","{'id': 1, 'name': 'Regular Play'}","{'id': 968, 'name': 'Arsenal WFC'}",0.0,NaN,[02403d23-9d60-4754-a1d7-7143ce0cffd7],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
events_df.shape

(1210151, 42)

In [ ]:
events_df.columns

Index(['id', 'index', 'period', 'timestamp', 'minute', 'second', 'type',
       'possession', 'possession_team', 'play_pattern', 'team', 'duration',
       'tactics', 'related_events', 'player', 'position', 'location', 'pass',
       'carry', 'under_pressure', 'ball_receipt', 'counterpress', 'duel',
       'dribble', 'goalkeeper', 'clearance', 'shot', 'foul_committed', 'out',
       'interception', 'off_camera', 'ball_recovery', 'foul_won', 'block',
       'substitution', '50_50', 'injury_stoppage', 'miscontrol',
       'bad_behaviour', 'half_start', 'half_end', 'player_off'],
      dtype='object')

En este caso estamos interesados en los tiros, por lo que debemos seleccionar solo estos eventos.

In [ ]:
events_df['shot'].notna().sum() #nos indica cuantos tiros tendremos disponibles 

9362

In [ ]:
all_shots = events_df[events_df['shot'].notna()] 

## Conociendo y eligiendo features del dataset *all_shots*

In [ ]:
all_shots.head(3)

,id,index,period,timestamp,minute,second,type,possession,possession_team,play_pattern,team,duration,tactics,related_events,player,position,location,pass,carry,under_pressure,ball_receipt,counterpress,duel,dribble,goalkeeper,clearance,shot,foul_committed,out,interception,off_camera,ball_recovery,foul_won,block,substitution,50_50,injury_stoppage,miscontrol,bad_behaviour,half_start,half_end,player_off
91,9076cf40-9179-432a-9663-94cbbff6ce74,92,1,2022-03-03 00:01:27.151,1,27,"{'id': 16, 'name': 'Shot'}",5,"{'id': 2647, 'name': 'Aston Villa'}","{'id': 8, 'name': 'From Keeper'}","{'id': 2647, 'name': 'Aston Villa'}",0.154537,NaN,"[0777e98e-8c69-4b42-9fe4-8f702d3b2c3b, 94c7ef5...","{'id': 10102, 'name': 'Stine Larsen'}","{'id': 24, 'name': 'Left Center Forward'}","[100.6, 54.6]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'statsbomb_xg': 0.037459332000000005, 'end_lo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277,b3a321bb-597c-49da-b039-21cf7eeb6095,278,1,2022-03-03 00:05:21.756,5,21,"{'id': 16, 'name': 'Shot'}",11,"{'id': 968, 'name': 'Arsenal WFC'}","{'id': 2, 'name': 'From Corner'}","{'id': 968, 'name': 'Arsenal WFC'}",0.698834,NaN,[02ac6aab-5cba-498f-a312-88852878e31d],"{'id': 15623, 'name': 'Vivianne Miedema'}","{'id': 21, 'name': 'Left Wing'}","[109.4, 43.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'statsbomb_xg': 0.026655061, 'end_location': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337,87d44b8c-955b-4341-a3c3-c6665fdb37bd,338,1,2022-03-03 00:06:57.808,6,57,"{'id': 16, 'name': 'Shot'}",14,"{'id': 968, 'name': 'Arsenal WFC'}","{'id': 4, 'name': 'From Throw In'}","{'id': 968, 'name': 'Arsenal WFC'}",0.234688,NaN,"[95b12535-a709-4b4f-8017-00d9c497b372, f828da8...","{'id': 10650, 'name': 'Jill Roord'}","{'id': 15, 'name': 'Left Center Midfield'}","[95.6, 38.9]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'statsbomb_xg': 0.034883816000000005, 'end_lo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
all_shots.shape

(9362, 42)

Con la ayuda de la documentación de statsbomb podemos empezar a discernir que variables son relevantes.

Podemos descartar las columnas: 
* `['id', 'index', 'possesion', 'duration']` que no aportan información relevante
* `['team', 'type']` cuya información la podemos encontrar dentro de la variable shot.
* ['second', 'related_events', 'timestamp'] por simplicidad de nuestro modelo 


Ahora, muchas de las columnas que hacen referencia a otros eventos tienen un valor de NaN, por lo que en un principio no tienen que ver con el evento de tiro. Aún así, analicemos si hay algún evento que sucede simúltaneo.

In [ ]:
other_events = ['tactics', 'pass', 'carry', 'under_pressure', 'ball_receipt', 'counterpress', 'duel', 'dribble', 'goalkeeper', 'clearance', 'foul_committed', 'out',
                'interception', 'off_camera', 'ball_recovery', 'foul_won', 'block', 'substitution', '50_50', 'injury_stoppage', 'miscontrol', 'bad_behaviour', 
                'half_start', 'half_end', 'player_off']
for event in other_events:
  print('El evento ' + event + ' tiene ' + str(all_shots[event].notna().sum()) + ' valores no vacíos')

El evento tactics tiene 0 valores no vacíos
El evento pass tiene 0 valores no vacíos
El evento carry tiene 0 valores no vacíos
El evento under_pressure tiene 1771 valores no vacíos
El evento ball_receipt tiene 0 valores no vacíos
El evento counterpress tiene 0 valores no vacíos
El evento duel tiene 0 valores no vacíos
El evento dribble tiene 0 valores no vacíos
El evento goalkeeper tiene 0 valores no vacíos
El evento clearance tiene 0 valores no vacíos
El evento foul_committed tiene 0 valores no vacíos
El evento out tiene 73 valores no vacíos
El evento interception tiene 0 valores no vacíos
El evento off_camera tiene 14 valores no vacíos
El evento ball_recovery tiene 0 valores no vacíos
El evento foul_won tiene 0 valores no vacíos
El evento block tiene 0 valores no vacíos
El evento substitution tiene 0 valores no vacíos
El evento 50_50 tiene 0 valores no vacíos
El evento injury_stoppage tiene 0 valores no vacíos
El evento miscontrol tiene 0 valores no vacíos
El evento bad_behaviour tie

Los eventos restantes que tienen algún valor distinto de NaN son `under_pressure`, `out` y `off_camera`.

*   `off_camera`: contiene TRUE si el evento ocurrió mientras la cámara estaba apagada.
*   `out`: contiene TRUE si el resultado del evento es la pelota saliendo de los límites.
*   `under_pressure`: contiene TRUE si la acción fue realizada mientras se tuvo la presión de un oponente.

Podemos descartar la columna off_camera, ya que si en dado caso de que este factor fuera determinante se podría ver más adelante con la falta de información. 

La variable out se puede obviar debido a que dentro de nuestro contexto lo que nos interesa es el resultado del tiro y este se encuentra dentro de la variable shot.





Ahora formemos un dataset con las columnas que consideramos relevantes

In [ ]:
all_shots=all_shots[['period', 'minute', 'possession_team', 'play_pattern', 'player', 'position', 'location', 'shot', 'under_pressure']]

In [ ]:
all_shots.head()

,period,minute,possession_team,play_pattern,player,position,location,shot,under_pressure
91,1,1,"{'id': 2647, 'name': 'Aston Villa'}","{'id': 8, 'name': 'From Keeper'}","{'id': 10102, 'name': 'Stine Larsen'}","{'id': 24, 'name': 'Left Center Forward'}","[100.6, 54.6]","{'statsbomb_xg': 0.037459332000000005, 'end_lo...",NaN
277,1,5,"{'id': 968, 'name': 'Arsenal WFC'}","{'id': 2, 'name': 'From Corner'}","{'id': 15623, 'name': 'Vivianne Miedema'}","{'id': 21, 'name': 'Left Wing'}","[109.4, 43.0]","{'statsbomb_xg': 0.026655061, 'end_location': ...",NaN
337,1,6,"{'id': 968, 'name': 'Arsenal WFC'}","{'id': 4, 'name': 'From Throw In'}","{'id': 10650, 'name': 'Jill Roord'}","{'id': 15, 'name': 'Left Center Midfield'}","[95.6, 38.9]","{'statsbomb_xg': 0.034883816000000005, 'end_lo...",NaN
459,1,10,"{'id': 968, 'name': 'Arsenal WFC'}","{'id': 4, 'name': 'From Throw In'}","{'id': 15619, 'name': 'Bethany Mead'}","{'id': 23, 'name': 'Center Forward'}","[95.6, 44.6]","{'statsbomb_xg': 0.020818915, 'end_location': ...",NaN
608,1,14,"{'id': 968, 'name': 'Arsenal WFC'}","{'id': 2, 'name': 'From Corner'}","{'id': 18999, 'name': 'Leah Williamson'}","{'id': 3, 'name': 'Right Center Back'}","[111.0, 33.7]","{'statsbomb_xg': 0.11266468, 'end_location': [...",NaN


In [ ]:
all_shots.shape

(9362, 9)

Como se puede observar hay columnas cuyos valores son diccionarios, vamos a extraer lo relevante de cada una de las columnas

In [ ]:
all_shots=all_shots.reset_index()

In [ ]:
#extraemos el valor de la llave name para possession_team
team_in_possession = []
for fila in range(all_shots.shape[0]):
  team_in_possession.append(all_shots['possession_team'][fila]['name'])

#extraemos el valor de la llave name para play_pattern
pattern_of_play=[]
for fila in range(all_shots.shape[0]):
  pattern_of_play.append(all_shots['play_pattern'][fila]['name'])

#extraemos el valor de la llave name para player
player_list=[]
for fila in range(all_shots.shape[0]):
  player_list.append(all_shots['player'][fila]['name'])

#extraemos el valor de la llave name para position
player_position=[]
for fila in range(all_shots.shape[0]):
  player_position.append(all_shots['position'][fila]['name'])

In [ ]:
#Agreguemos las listas con los valores deseados al dataframe
all_shots['player_name']=player_list
all_shots['pattern_of_play']=pattern_of_play
all_shots['team']=team_in_possession
all_shots['player_position']=player_position

In [ ]:
all_shots.head()

,index,period,minute,possession_team,play_pattern,player,position,location,shot,under_pressure,player_name,pattern_of_play,team,player_position
0,91,1,1,"{'id': 2647, 'name': 'Aston Villa'}","{'id': 8, 'name': 'From Keeper'}","{'id': 10102, 'name': 'Stine Larsen'}","{'id': 24, 'name': 'Left Center Forward'}","[100.6, 54.6]","{'statsbomb_xg': 0.037459332000000005, 'end_lo...",NaN,Stine Larsen,From Keeper,Aston Villa,Left Center Forward
1,277,1,5,"{'id': 968, 'name': 'Arsenal WFC'}","{'id': 2, 'name': 'From Corner'}","{'id': 15623, 'name': 'Vivianne Miedema'}","{'id': 21, 'name': 'Left Wing'}","[109.4, 43.0]","{'statsbomb_xg': 0.026655061, 'end_location': ...",NaN,Vivianne Miedema,From Corner,Arsenal WFC,Left Wing
2,337,1,6,"{'id': 968, 'name': 'Arsenal WFC'}","{'id': 4, 'name': 'From Throw In'}","{'id': 10650, 'name': 'Jill Roord'}","{'id': 15, 'name': 'Left Center Midfield'}","[95.6, 38.9]","{'statsbomb_xg': 0.034883816000000005, 'end_lo...",NaN,Jill Roord,From Throw In,Arsenal WFC,Left Center Midfield
3,459,1,10,"{'id': 968, 'name': 'Arsenal WFC'}","{'id': 4, 'name': 'From Throw In'}","{'id': 15619, 'name': 'Bethany Mead'}","{'id': 23, 'name': 'Center Forward'}","[95.6, 44.6]","{'statsbomb_xg': 0.020818915, 'end_location': ...",NaN,Bethany Mead,From Throw In,Arsenal WFC,Center Forward
4,608,1,14,"{'id': 968, 'name': 'Arsenal WFC'}","{'id': 2, 'name': 'From Corner'}","{'id': 18999, 'name': 'Leah Williamson'}","{'id': 3, 'name': 'Right Center Back'}","[111.0, 33.7]","{'statsbomb_xg': 0.11266468, 'end_location': [...",NaN,Leah Williamson,From Corner,Arsenal WFC,Right Center Back


In [ ]:
#procedamos a quitar las columnas que tienen valores dict que ya tratamos
all_shots = all_shots.drop(['index', 'possession_team', 'play_pattern', 'player', 'position'], axis=1)

In [ ]:
all_shots.head()

,period,minute,location,shot,under_pressure,player_name,pattern_of_play,team,player_position
0,1,1,"[100.6, 54.6]","{'statsbomb_xg': 0.037459332000000005, 'end_lo...",NaN,Stine Larsen,From Keeper,Aston Villa,Left Center Forward
1,1,5,"[109.4, 43.0]","{'statsbomb_xg': 0.026655061, 'end_location': ...",NaN,Vivianne Miedema,From Corner,Arsenal WFC,Left Wing
2,1,6,"[95.6, 38.9]","{'statsbomb_xg': 0.034883816000000005, 'end_lo...",NaN,Jill Roord,From Throw In,Arsenal WFC,Left Center Midfield
3,1,10,"[95.6, 44.6]","{'statsbomb_xg': 0.020818915, 'end_location': ...",NaN,Bethany Mead,From Throw In,Arsenal WFC,Center Forward
4,1,14,"[111.0, 33.7]","{'statsbomb_xg': 0.11266468, 'end_location': [...",NaN,Leah Williamson,From Corner,Arsenal WFC,Right Center Back


Dentro de `shot` existe información relevante. 

La lista de todos los valores que podrían aparecer se encuentra en este [link](https://github.com/statsbomb/open-data/blob/master/doc/Open%20Data%20Events%20v4.0.0.pdf). 

En este caso se van a extraer las variables: `technique,` `body_part`, `type`, `outcome`, `first_time`, `statsbomb_xg`.

Nota: statsbomb tiene su propio modelo de goles esperados, en dado caso podríamos comparar los resultados de nuestro modelo vs el modelo de statsbomb

In [ ]:
technique = []
body_part = []
tipe = [] 
outcome = []
statsbomb_xg = []
for fila in range(all_shots.shape[0]):
  technique.append(all_shots['shot'][fila]['technique']['name'])
  body_part.append(all_shots['shot'][fila]['body_part']['name'])
  tipe.append(all_shots['shot'][fila]['type']['name'])
  outcome.append(all_shots['shot'][fila]['outcome']['name'])
  statsbomb_xg.append(all_shots['shot'][fila]['statsbomb_xg'])

In [ ]:
#se extrae first_time
first_time=[]
for fila in range(all_shots.shape[0]):
  try: 
    first_time.append(all_shots['shot'][fila]['first_time'])
  except:
    first_time.append(False)

In [ ]:
#agreguemos estos nuevos features a nuestro dataframe
all_shots['statsbomb_xg']=statsbomb_xg
all_shots['technique']=technique
all_shots['body_part']=body_part
all_shots['tipe']=tipe
all_shots['outcome']=outcome
all_shots['first_time']=first_time

In [ ]:
all_shots.head()

,period,minute,location,shot,under_pressure,player_name,pattern_of_play,team,player_position,statsbomb_xg,technique,body_part,tipe,outcome,first_time
0,1,1,"[100.6, 54.6]","{'statsbomb_xg': 0.037459332000000005, 'end_lo...",NaN,Stine Larsen,From Keeper,Aston Villa,Left Center Forward,0.037459,Normal,Left Foot,Open Play,Blocked,False
1,1,5,"[109.4, 43.0]","{'statsbomb_xg': 0.026655061, 'end_location': ...",NaN,Vivianne Miedema,From Corner,Arsenal WFC,Left Wing,0.026655,Normal,Head,Open Play,Wayward,False
2,1,6,"[95.6, 38.9]","{'statsbomb_xg': 0.034883816000000005, 'end_lo...",NaN,Jill Roord,From Throw In,Arsenal WFC,Left Center Midfield,0.034884,Normal,Right Foot,Open Play,Blocked,False
3,1,10,"[95.6, 44.6]","{'statsbomb_xg': 0.020818915, 'end_location': ...",NaN,Bethany Mead,From Throw In,Arsenal WFC,Center Forward,0.020819,Normal,Right Foot,Open Play,Off T,False
4,1,14,"[111.0, 33.7]","{'statsbomb_xg': 0.11266468, 'end_location': [...",NaN,Leah Williamson,From Corner,Arsenal WFC,Right Center Back,0.112665,Half Volley,Left Foot,Open Play,Off T,True


In [ ]:
#ya no necesitamos la columna shot, por lo que podemos sacarla del dataframe
all_shots = all_shots.drop('shot', axis=1)

In [ ]:
all_shots.head()

,period,minute,location,under_pressure,player_name,pattern_of_play,team,player_position,statsbomb_xg,technique,body_part,tipe,outcome,first_time
0,1,1,"[100.6, 54.6]",NaN,Stine Larsen,From Keeper,Aston Villa,Left Center Forward,0.037459,Normal,Left Foot,Open Play,Blocked,False
1,1,5,"[109.4, 43.0]",NaN,Vivianne Miedema,From Corner,Arsenal WFC,Left Wing,0.026655,Normal,Head,Open Play,Wayward,False
2,1,6,"[95.6, 38.9]",NaN,Jill Roord,From Throw In,Arsenal WFC,Left Center Midfield,0.034884,Normal,Right Foot,Open Play,Blocked,False
3,1,10,"[95.6, 44.6]",NaN,Bethany Mead,From Throw In,Arsenal WFC,Center Forward,0.020819,Normal,Right Foot,Open Play,Off T,False
4,1,14,"[111.0, 33.7]",NaN,Leah Williamson,From Corner,Arsenal WFC,Right Center Back,0.112665,Half Volley,Left Foot,Open Play,Off T,True


Con esto hemos concluido la sección de carga de datos, por lo que guardaremos el dataframe y continuaremos en un nuevo notebook

In [ ]:
all_shots.to_csv('shots_carga.csv')